In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

Наши новости

In [153]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [154]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [155]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [156]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel
import gensim
import pymorphy2  # pip install pymorphy2
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [157]:
#nltk.download('stopwords')
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

morph = pymorphy2.MorphAnalyzer()

151


In [158]:
with open('stopwords.txt', encoding="utf-8") as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)
print(stopword_ru)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [159]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [160]:
print(news.head(5))
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)


   doc_id                                              title
0       6  Заместитель председателяnправительства РФnСерг...
1    4896  Матч 1/16 финала Кубка России по футболу был п...
2    4897  Форвард «Авангарда» Томаш Заборский прокоммент...
3    4898  Главный тренер «Кубани» Юрий Красножан прокомм...
4    4899  Решением попечительского совета владивостокско...
UsageError: Line magic function `%%time` not found.


In [161]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

news.head(5)

Wall time: 5min 4s


,doc_id,title
0,6,"[заместитель, председатель, правительство, рф,..."
1,4896,"[матч, 1/16, финал, кубок, россия, футбол, при..."
2,4897,"[форвард, авангард, томаш, заборский, прокомме..."
3,4898,"[главный, тренер, кубань, юрий, красножанин, п..."
4,4899,"[решение, попечительский, совет, владивостокск..."


А теперь в 3 строчки обучим нашу модель

In [162]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]
# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [163]:
common_dictionary[1]
#common_corpus

'алексей'

Все просто - это словарь наших слов

Запускаем обучение

In [164]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min 2s


In [165]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [166]:
lda.print_topics()

[(16,
  '0.048*"фестиваль" + 0.033*"вице-премьер" + 0.024*"снять" + 0.022*"бомба" + 0.019*"мероприятие" + 0.017*"лётчик" + 0.016*"ремонт" + 0.014*"мэй" + 0.013*"образовать" + 0.013*"программа"'),
 (23,
  '0.028*"инвестиция" + 0.021*"белый" + 0.016*"умереть" + 0.012*"сенатор" + 0.010*"жизнь" + 0.010*"сша" + 0.009*"саммит" + 0.009*"мировой" + 0.008*"правовой" + 0.007*"первый"'),
 (8,
  '0.039*"чёрный" + 0.034*"китайский" + 0.028*"китай" + 0.022*"вода" + 0.015*"риа" + 0.015*"санкт-петербург" + 0.015*"медицина" + 0.014*"полоса" + 0.012*"мвд" + 0.010*"постановление"'),
 (4,
  '0.023*"остров" + 0.014*"высота" + 0.013*"запуск" + 0.012*"сообщество" + 0.011*"n—" + 0.011*"гора" + 0.011*"память" + 0.011*"северный" + 0.010*"первый" + 0.009*"явление"'),
 (11,
  '0.085*"вероятно" + 0.060*"озеро" + 0.049*"600" + 0.032*"калифорния" + 0.027*"приземлиться" + 0.026*"комплексный" + 0.026*"дания" + 0.023*"зеландия" + 0.018*"разногласие" + 0.015*"ум"'),
 (22,
  '0.013*"писать" + 0.010*"женщина" + 0.010*"рын

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [167]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(3, 0.1157299),
 (8, 0.055382933),
 (14, 0.5429492),
 (19, 0.07689899),
 (24, 0.18728457)]

In [168]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: млн составить рост рубль 2017 рынок цена
topic_1: украина украинский операция сторона киев палата норма
topic_2: место первый экипаж двигатель скорость км второй
topic_3: исследование 2019 взрыв пациент конкурс препарат красный
topic_4: остров высота запуск сообщество n— гора память
topic_5: млрд банк университет ставка рубль налог цена
topic_6: россия газ российский фонд ракета сша рубль
topic_7: произойти тело обнаружить район мужчина пострадать статья
topic_8: чёрный китайский китай вода риа санкт-петербург медицина
topic_9: восток медведев северный лодка польша сценарий польский
topic_10: сша американский поток российский россия всё эксперт
topic_11: вероятно озеро 600 калифорния приземлиться комплексный дания
topic_12: россия новый проект научный система москва российский
topic_13: военный министерство пенсия com академия боевой космонавт
topic_14: n— век товар очень хороший дональд первый
topic_15: гражданин погибнуть россия российский население ... 2017
topic_16: фестив

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [169]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, common_corpus, common_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.189856 -0.003090       1        1  17.267722
6     -0.168153 -0.014431       2        1  14.664005
15    -0.157716 -0.004733       3        1   7.988030
20    -0.147667  0.076589       4        1   6.778009
22    -0.151216 -0.043548       5        1   6.270900
7     -0.100643  0.092821       6        1   6.150814
10    -0.129819  0.038855       7        1   5.149017
3     -0.044717  0.012539       8        1   4.158955
17    -0.105860  0.019321       9        1   4.058936
14    -0.058816 -0.030464      10        1   3.958652
1     -0.083293  0.028732      11        1   3.759671
13    -0.046155  0.100418      12        1   2.552020
2      0.006300 -0.023486      13        1   2.442435
24    -0.066222  0.002041      14        1   2.238277
21     0.019919  0.017666      15        1   2.127292
0      0.086407 -0.298119      16        1   2.004391
23     0.042695  0.047506      17        1   1.615794
5      0.026554 -0.274979      18        1   1.608942
4     -0.016339  0.023985      19        1   1.475401
16     0.194400  0.028001      20        1   1.003828
8      0.105302  0.044374      21        1   0.826722
19     0.155289 -0.017276      22        1   0.704678
18     0.246947  0.058498      23        1   0.541288
9      0.228805  0.074863      24        1   0.489094
11     0.353852  0.043918      25        1   0.165128, topic_info=                 Term          Freq         Total Category  logprob  loglift
9725     исследование  15467.000000  15467.000000  Default  30.0000  30.0000
1251              млн  16773.000000  16773.000000  Default  29.0000  29.0000
6097             млрд  12220.000000  12220.000000  Default  28.0000  28.0000
11944         военный  13200.000000  13200.000000  Default  27.0000  27.0000
294           украина  20054.000000  20054.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
12536        поражать     38.598710    391.753810  Topic25  -5.6900   4.0888
3935             мина     32.697533    463.685685  Topic25  -5.8560   3.7543
2605         заявлять     41.738836   2999.727242  Topic25  -5.6118   2.1314
13338          статья     38.111746   9520.258461  Topic25  -5.7027   0.8856
2970   предполагаться     29.910094   2435.797304  Topic25  -5.9451   2.0064

[1713 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
2065      1  0.041723    ...
2065      3  0.302806    ...
2065      4  0.132991    ...
2065      5  0.061399    ...
2065      6  0.045753    ...
...     ...       ...    ...
7760     15  0.000861  яркий
7760     17  0.038742  яркий
7760     18  0.009470  яркий
7760     20  0.009470  яркий
7760     21  0.001722  яркий

[7644 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 7, 16, 21, 23, 8, 11, 4, 18, 15, 2, 14, 3, 25, 22, 1, 24, 6, 5, 17, 9, 20, 19, 10, 12])

In [170]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [171]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.0,0.000000,0.000000,0.023078,0.182926,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.057206,0.0,0.000000,0.0,0.612431,0.000000
1,4896,0.0,0.0,0.0,0.714283,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.057832,0.0,0.000000,0.000000
2,4897,0.0,0.0,0.0,0.115727,0.000000,0.000000,0.000000,0.0,0.055386,...,0.0,0.0,0.0,0.0,0.076870,0.0,0.000000,0.0,0.000000,0.187249
3,4898,0.0,0.0,0.0,0.000000,0.431605,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.120134,0.0,0.000000,0.0,0.000000,0.000000
4,4899,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.706615,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.209485,0.057861


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [172]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [173]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [174]:
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.0869618 , 0.        ,
       0.05986562, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.22711346, 0.        , 0.10985298,
       0.07707598, 0.        , 0.27928048, 0.        , 0.        ,
       0.        , 0.        , 0.05965425, 0.06731935, 0.02201754])

In [175]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [176]:
get_user_embedding(user_articles_list)

array([0.00728848, 0.00980299, 0.01660089, 0.03836575, 0.00242516,
       0.        , 0.12726741, 0.03605221, 0.05227002, 0.01256909,
       0.07082941, 0.        , 0.19366719, 0.04043073, 0.00177194,
       0.14644955, 0.        , 0.0403054 , 0.02614734, 0.        ,
       0.02626506, 0.04341123, 0.06396165, 0.00478925, 0.02779135])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

In [177]:
#users['articles'].iloc[33]

In [178]:
#" ".join(news[news['doc_id']==323186]['title'].iloc[0])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [179]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.016213,0.096662,0.062261,0.045772,0.005253,0.009978,0.037025,0.025636,0.006379,...,0.039159,0.073899,0.086502,0.0,0.000000,0.030339,0.004272,0.150471,0.017553,0.003670
1,u108690,0.021818,0.109615,0.000000,0.024468,0.007807,0.000000,0.109514,0.055901,0.028037,...,0.088195,0.005273,0.028432,0.0,0.002818,0.019147,0.023257,0.070765,0.021159,0.029722
2,u108339,0.008274,0.035876,0.000000,0.060174,0.005270,0.010390,0.098282,0.127578,0.047551,...,0.114740,0.000000,0.038267,0.0,0.005743,0.043907,0.053605,0.027844,0.026413,0.013336


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [180]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [181]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.016213,0.096662,0.062261,0.045772,0.005253,0.009978,0.037025,0.025636,0.006379,...,0.073899,0.086502,0.0,0.000000,0.030339,0.004272,0.150471,0.017553,0.003670,0
1,u108690,0.021818,0.109615,0.000000,0.024468,0.007807,0.000000,0.109514,0.055901,0.028037,...,0.005273,0.028432,0.0,0.002818,0.019147,0.023257,0.070765,0.021159,0.029722,1
2,u108339,0.008274,0.035876,0.000000,0.060174,0.005270,0.010390,0.098282,0.127578,0.047551,...,0.000000,0.038267,0.0,0.005743,0.043907,0.053605,0.027844,0.026413,0.013336,1


In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [183]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [184]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [185]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.12406163, 0.0128019 , 0.43323496, 0.23497662, 0.04436297,
       0.05582541, 0.21140373, 0.01932031, 0.09301412, 0.20132553])

In [186]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [187]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.214157, F-Score=0.631, Precision=0.520, Recall=0.804


In [188]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [189]:
#мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

Confusion matrix, without normalization
[[1573  182]
 [  49  196]]


NameError: name 'itertools' is not defined

In [113]:
roc_auc_score(y_test, preds)

0.9507924879353452

В целом мы видим, что получившиеся векторные представления содержат какой-то сигнал и позволяют решать нашу прикладную задачу. 

### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

#### Ссылки

1. http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf
2. https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

#### Библиотеки, которые нужно установить:

1. gensim
2. razdel
3. pymorphy2
4. nltk